<h1 align="center"><font size="6">Qualidade Vinho Tinto</font></h1>

Nesse trabalho, será desenvolvido um modelo capaz de classificar a qualidade de um vinho tinto atráves de modelo de regressão e de árvore de decisão.

### Importando os dados e bibliotecas

Dados importados foram fornecidos peolo Kaggle em https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009/data

In [66]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_similarity_score

In [67]:
wine_df = pd.read_csv("winequality-red.csv", sep =',')
wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


<h1 align="center"><font size="6"> Regressão Logistica</font></h1>

Enquanto a Regressão Linear é adequada para estimar valores contínuos (por exemplo, estimar o preço de um imóvel), não é a melhor ferramenta para prever a classe de um ponto de dados observado. Para estimar a classe de um determinado dado, precisamos de algum tipo de orientação sobre a qual seria a **classe mais provável** para esse dado. Para isso, usamos **Regressão Logística**.

<div class="alert alert-success alertsuccess" style="margin-top: 20px">
<font size = 3><strong>Relembrando a Regressão Linear:</strong></font>
<br>
<br>
    Como você sabe, a <b>regressão linear</b> encontra uma função que relaciona uma variável dependente contínua, <b>y</b>, a alguns preditores (variáveis independentes $x_1$, $x_2$, etc.). Por exemplo, a regressão linear simples assume uma função da forma:
<br><br>
$$
y = \theta_0 + \theta_1  x_1 + \theta_2  x_2 + \cdots
$$
<br>
e encontra os valores dos parâmetros $\theta_0, \theta_1, \theta_2$, etc, onde o termo $\theta_0$ é o "intercepto". Pode ser geralmente mostrado como:
<br><br>
$$
ℎ_\theta(𝑥) = \theta^TX
$$
<p></p>

</div>
A **Regressão Logística** é uma variação da Regressão Linear. É útil quando a variável dependente observada, <b>y</b>, é categórica. Ela produz uma fórmula que prevê a probabilidade do rótulo da classe como uma função das variáveis independentes.

A regressão logística ajusta-se a uma curva especial em forma de s tomando a regressão linear e transformando a estimativa numérica em uma probabilidade com a seguinte função, denominada função sigmoide 𝜎:
$$
ℎ_\theta(𝑥) = \sigma({\theta^TX}) =  \frac {e^{(\theta_0 + \theta_1  x_1 + \theta_2  x_2 +...)}}{1 + e^{(\theta_0 + \theta_1  x_1 + \theta_2  x_2 +\cdots)}}
$$
Ou:
$$
ProbabilidadeDaClasse_1 =  P(Y=1|X) = \sigma({\theta^TX}) = \frac{e^{\theta^TX}}{1+e^{\theta^TX}} 
$$

Nesta equação, ${\theta^TX}$ é o resultado da regressão (a soma das variáveis ponderadas pelos coeficientes), 'exp' é a função exponencial e $\sigma(\theta^TX)$ é a função sigmóide ou [logística] (http: //en.wikipedia .org / wiki / Logistic_function), também chamado de curva logística, e possui o formato de um "S" (curva sigmóide).

Então, resumidamente, a regressão logística passa a entrada através da função logística/sigmoide, mas depois trata o resultado como uma probabilidade:

<img
src="https://ibm.box.com/shared/static/kgv9alcghmjcv97op4d6onkyxevk23b1.png" width="400" align="center">

O objetivo do algoritmo de __Regressão Logística__, é encontrar os melhores parâmetros θ, para $ ℎ_\theta(𝑥)$ = $\sigma({\theta^TX})$, de tal forma que o modelo prediga melhor a classe de cada caso. 

#### Material retirado da própria aula, para melhor exemplificar

In [68]:
wine_df = wine_df[['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'pH', 'density', 'quality']]
wine_df['quality'] = wine_df['quality'].astype('int')
wine_df.head()

,fixed acidity,volatile acidity,residual sugar,chlorides,free sulfur dioxide,pH,density,quality
0,7.4,0.70,1.9,0.076,11.0,3.51,0.9978,5
1,7.8,0.88,2.6,0.098,25.0,3.20,0.9968,5
2,7.8,0.76,2.3,0.092,15.0,3.26,0.9970,5
3,11.2,0.28,1.9,0.075,17.0,3.16,0.9980,6
4,7.4,0.70,1.9,0.076,11.0,3.51,0.9978,5


In [69]:
X = np.asarray(wine_df[['fixed acidity', 'volatile acidity', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'pH', 'density']])
X[0:10]

array([[ 7.4   ,  0.7   ,  1.9   ,  0.076 , 11.    ,  3.51  ,  0.9978],
       [ 7.8   ,  0.88  ,  2.6   ,  0.098 , 25.    ,  3.2   ,  0.9968],
       [ 7.8   ,  0.76  ,  2.3   ,  0.092 , 15.    ,  3.26  ,  0.997 ],
       [11.2   ,  0.28  ,  1.9   ,  0.075 , 17.    ,  3.16  ,  0.998 ],
       [ 7.4   ,  0.7   ,  1.9   ,  0.076 , 11.    ,  3.51  ,  0.9978],
       [ 7.4   ,  0.66  ,  1.8   ,  0.075 , 13.    ,  3.51  ,  0.9978],
       [ 7.9   ,  0.6   ,  1.6   ,  0.069 , 15.    ,  3.3   ,  0.9964],
       [ 7.3   ,  0.65  ,  1.2   ,  0.065 , 15.    ,  3.39  ,  0.9946],
       [ 7.8   ,  0.58  ,  2.    ,  0.073 ,  9.    ,  3.36  ,  0.9968],
       [ 7.5   ,  0.5   ,  6.1   ,  0.071 , 17.    ,  3.35  ,  0.9978]])

In [70]:
y = np.asarray(wine_df['quality'])
y [0:10]

array([5, 5, 5, 6, 5, 5, 5, 7, 7, 5])

In [71]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:10]

array([[-0.52835961,  0.96187667, -0.45321841, -0.24370669, -0.46619252,
         1.28864292,  0.55827446],
       [-0.29854743,  1.96744245,  0.04341614,  0.2238752 ,  0.87263823,
        -0.7199333 ,  0.02826077],
       [-0.29854743,  1.29706527, -0.16942723,  0.09635286, -0.08366945,
        -0.33117661,  0.13426351],
       [ 1.65485608, -1.38444349, -0.45321841, -0.26496041,  0.10759209,
        -0.97910442,  0.6642772 ],
       [-0.52835961,  0.96187667, -0.45321841, -0.24370669, -0.46619252,
         1.28864292,  0.55827446],
       [-0.52835961,  0.73841761, -0.5241662 , -0.26496041, -0.27493098,
         1.28864292,  0.55827446],
       [-0.24109439,  0.40322901, -0.66606179, -0.39248274, -0.08366945,
        -0.07200549, -0.18374471],
       [-0.58581266,  0.68255284, -0.94985296, -0.47749763, -0.08366945,
         0.51112954, -1.13776936],
       [-0.29854743,  0.29149948, -0.38227061, -0.30746785, -0.65745405,
         0.3167512 ,  0.02826077],
       [-0.47090657, -0.1554

In [72]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Dados de Treinamento:', X_train.shape,  y_train.shape)
print ('Dados de Validação:', X_test.shape,  y_test.shape)

Dados de Treinamento: (1279, 7) (1279,)
Dados de Validação: (320, 7) (320,)


In [73]:
RL = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)

In [74]:
yhat = RL.predict(X_test)

In [75]:
yhat_prob = RL.predict_proba(X_test)

In [76]:
RL
yhat
yhat_prob

array([[0.08890789, 0.07908379, 0.34834142, 0.27083123, 0.12514658,
        0.08768909],
       [0.0974459 , 0.11742246, 0.3134779 , 0.26303786, 0.11517289,
        0.093443  ],
       [0.08614093, 0.09639204, 0.1497753 , 0.33181888, 0.2325844 ,
        0.10328846],
       ...,
       [0.09770227, 0.11676604, 0.33450995, 0.26205452, 0.09688765,
        0.09207956],
       [0.10426838, 0.14210766, 0.41669808, 0.19614087, 0.05689399,
        0.08389103],
       [0.101403  , 0.13396593, 0.21065913, 0.28717602, 0.15913497,
        0.10766095]])

In [77]:
jaccard_similarity_score(y_test, yhat)

0.584375